In [1]:
import glob

from fastai.text import *
# from fastai.callbacks import SaveModelCallback
import sentencepiece as spm #https://github.com/google/sentencepiece

from prod.sp_tok import *

torch.backends.cudnn.enabled=False #for quant.313v machine.  Not sure why...

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

In [2]:
PATH = Path('./data/rnn')
sp_model = './all_tweets_es_0606'

defaults.text_spec_tok.append(NL) #add a New Line special char
sp_vocab = Vocab( get_itos(sp_model) )
mycust_tok = CustomTokenizer(SPTokenizer,sp_model,pre_rules=default_rules)

In [3]:
test_df = pd.read_csv('./data/haha_2019_test.csv')
test_df['is_humor'] = 0
test_df['is_humor'].iloc[0] = 1 #create two classes in data.
## apply the rules
raw_text = test_df.loc[:,'text']

print("Default Rules:\n",[x.__name__ for x in default_rules],"\n\n")
for rule in default_rules:
    raw_text = raw_text.apply(lambda x: rule(str(x)))    
test_df['new_text'] =  raw_text #tokenizer adds xxbos

data_tst = TextClasDataBunch.from_df(PATH, test_df, test_df,
                               tokenizer=mycust_tok, vocab=sp_vocab,
                               text_cols='new_text', label_cols='is_humor')

In [4]:
model_wt_fnames = ['qrnn_F1_metric_90_dropout_0606.pth']
len(model_wt_fnames)

1

In [5]:
ensemble_preds = []
for wt_fname in model_wt_fnames:
    config = awd_lstm_clas_config.copy()
    config['qrnn'] = True
    config['n_hid'] = 2304
    
    learn = text_classifier_learner(data_tst, AWD_LSTM, drop_mult=0.7,pretrained=False,
                               config=config)
    learn = learn.load(wt_fname.split('.')[0]) #load weights
    preds_t,y_t,losses_t = learn.get_preds(DatasetType.Valid,with_loss=True, ordered=True)
    ensemble_preds.append(to_np(preds_t))

In [6]:
np.array(ensemble_preds).shape

(1, 6000, 2)

In [7]:
np.array(ensemble_preds)

array([[[0.23557 , 0.76443 ],
        [0.836961, 0.163039],
        [0.101897, 0.898103],
        [0.963748, 0.036252],
        ...,
        [0.452062, 0.547938],
        [0.986468, 0.013532],
        [0.359344, 0.640656],
        [0.21985 , 0.78015 ]]], dtype=float32)

In [8]:
ens_mean = np.array(ensemble_preds).mean(axis=0)
ens_mean

array([[0.23557 , 0.76443 ],
       [0.836961, 0.163039],
       [0.101897, 0.898103],
       [0.963748, 0.036252],
       ...,
       [0.452062, 0.547938],
       [0.986468, 0.013532],
       [0.359344, 0.640656],
       [0.21985 , 0.78015 ]], dtype=float32)

In [9]:
out_df = test_df['id'].to_frame().copy()
out_df['is_humor'] = ens_mean.argmax(axis=1)
out_df['funniness_average'] = 3.

In [11]:
out_df.head(10)

,id,is_humor,funniness_average
0,tweet1,1,3.0
1,tweet2,0,3.0
2,tweet3,1,3.0
3,tweet4,0,3.0
4,tweet5,0,3.0
5,tweet6,1,3.0
6,tweet7,0,3.0
7,tweet8,0,3.0
8,tweet9,1,3.0
9,tweet10,0,3.0


In [12]:
out_df.to_csv('sub0606_3_single.csv',header=True,index=False)